## TODO

## Introduction: Injustice at Work

Our data source explores the relationship between attributes of the complainants/complaints regarding Employee Discrimination charges and the outcomes of each charge.

Due to working on our personal machines, we chose 25,000 rows of data at random to represent the full dataset. The original dataset can be found here: https://github.com/PublicI/employment-discrimination/blob/master/data/complaints_10.txt

According to the Injustice at Work Center, each attribute is defined as follows:

- Unique ID: unique identifier for each case (a case is a collection of related charges)
- State Code: complainant state
- No of Employees Code: code indicating the approximate number of employees working for respondent employer
- No of Employees: approximate number of employees working for respondent employer
- NAICS Code: North American Industry Classification System code of respondent employer
- NAICS Description: North American Industry Classification System description of respondent company (e.g., crude petroleum and natural gas extraction)
- Institution Type Code: classification code of respondent employer
- Institution Type: classification of respondent employer (e.g., private employer)
- CP Date of Birth: complainant’s date of birth
- CP Sex: complainant’s gender
- Date First Office: date charge was filed
- Date FEPA Sent to EEOC: date charge was forwarded to the EEOC
- Closure Date: date investigation of case was closed
- Closure Code: code indicating how case was closed
- Closure Type: description indicating how case was closed (e.g., no cause finding issued)
- Monetary Benefits: monetary benefit complainant received
- Statute Code: code for statute under which charge was filed
- Statute: statute under which charge was filed (e.g., Americans with Disabilities Act)
- Basis Code: code for basis of discrimination
- Basis: basis of discrimination (e.g., race-black/African American)
- Issue Code: type code for adverse action alleged by complainant
- Issue: adverse action alleged by complainant (e.g., harassment)
- Court Filing Date: date complainant filed a lawsuit
- Civil Action Number: case number of lawsuit
- Court: court in which lawsuit was filed
- Litigation Resolution Date: date lawsuit was resolved
- Litigation Monetary Benefits: monetary damages recovered through lawsuit
- Litigation Case Type: case type of lawsuit

Our analysis will be looking to classify data by "Closure Code"(? or type), and we have deduced that the possible predictive attributes are as follows:
- State Code: complainant state
- No of Employees Code: code indicating the approximate number of employees working for respondent employer
- NAICS Code: North American Industry Classification System code of respondent employer
- Institution Type Code: classification code of respondent employer
- CP Date of Birth: complainant’s date of birth *
- CP Sex: complainant’s gender
- Date First Office: date charge was filed *
- Date FEPA Sent to EEOC: date charge was forwarded to the EEOC *
- Basis Code: code for basis of discrimination
- Issue Code: type code for adverse action alleged by complainant
- Litigation Case Type: case type of lawsuit

## Grouping of Classes

In the full dataset, the feature closure_code had 13 possible codes that described the outcome of each complaint. We needed to group these into two distinct classes for the purposes of our data.  We grouped these 13 codes into representing class 0 (not in favor of complainant), class 1 (in favor of complainants), or belonging to neither of these classes because it did not end clearly in favor of either group. If the case did not end clearly in favor of the complainant or employer based on the content of the case (i.e legal reasons like no jurisdiction for EEOC, case filed outside of the correct time range, etc), we labelled this as class 3 and did not include the cases in our analysis. 

We determined this grouping by using the following guide to show explanations of the codes meaning: https://www.umass.edu/eeodatanet/sites/default/files/IMS%20Codes.pdf
Further explanation on terminology can be found here: https://www.eeoc.gov/eeoc/statistics/enforcement/definitions.cfm)

Class 0 (in favor of employer) codes: 
- M3 - No cause finding issued
- M5 - Conciliation failure
- X7 - Closed due to court decision

Class 1 (in favor of employee) codes:
- M1 - Withdrawal with benefits
- M2 - Settlement with Benefits
- M4 - Successful Conciliation
- X1 - CP Filed Suit

Codes that fit into neither category (class 3, but will be discarded for analysis):
- X3 - Respondent Bankruptcy
- X9 - ADEA Sect 7(D) Closure
- Y1 - No Jurisdiction
- N2 - NRTS issues at CP Request (a Right To Sue letter is issued when the agency cannot determine whether the employer discriminated against an employee)
- Y2 - CP Withdrawal - No benefits
- X2 - Administrative closure

## Pandas Settings

In [56]:
import pandas as pd

pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 100)
pd.options.display.float_format = "{:,.2f}".format

## Preprocessing:
We are taking 25,000 rows from our dataset which included around 343,000 rows due to the limitations of doing this project on personal machines.

In [57]:
# code below was used to obtain a sample of 40,000 rows from the dataset.
'''
data = pd.read_csv("complaints_10.txt", sep="\t", skiprows=1,
                      dtype={1: str},
                      names=["unique_id", "state_code", "num_employee_code", "num_employees",
                             "naics_code", "naics_desc", "inst_type_code", "inst_type",
                             "birth_date", "sex", "date_filed", "date_sent_eeoc", "date_closed",
                             "closure_code", "closure_action", "monetary_benefits", "statute_code",
                             "statute", "basis_code", "basis", "issue_code", "issue",
                             "court_filing_date", "civil_action_num", "court", "resolution_date",
                             "litigation_monetary_benefits", "litigation_case_type"])

cols_to_drop = ['unique_id', 'num_employees', 'naics_desc', 
                'inst_type', 'date_closed', "closure_action",
                "monetary_benefits", "statute_code", "statute",
                "basis", "issue", "court_filing_date", "date_sent_eeoc",
                "civil_action_num", "court", "resolution_date",
                "litigation_monetary_benefits"
                ]

data = data.drop(cols_to_drop, axis = 1)
data = data.sample(n = 25_000)
data.to_csv('complaints_10-subset.csv', index=False)
'''
data = pd.read_csv('complaints_10-subset.csv')
data.head()

,state_code,num_employee_code,naics_code,inst_type_code,birth_date,sex,date_filed,closure_code,basis_code,issue_code,litigation_case_type
0,TX,C,nan,E,05/28/71,F,06/09/10,M3,GF,S5,NaN
1,NV,D,nan,E,04/18/66,NaN,05/11/10,M3,AC,D2,NaN
2,MA,NaN,"446,110.00",E,03/17/81,F,02/10/10,M3,OR,W1,NaN
3,PA,B,nan,G,07/18/56,F,11/03/09,M3,TP,T2,NaN
4,CA,A,nan,G,12/14/61,M,03/17/10,M3,VB,R6,NaN


Getting age from birth date and cleaning the age column

In [58]:
def get_age(dob, filing_date):
    if pd.isna(dob):
        return 0
    filing_year = int(str(filing_date)[-2:]) + 2000
    filing_year = filing_year if filing_year <= 2010 else og_year - 100
    birth_year = int(str(dob)[-2:]) + 2000
    birth_year = birth_year if birth_year <= 2010 else birth_year - 100
    age = filing_year - birth_year
    return age if age > 16 else 0

def get_year(date):
    if pd.isna(date):
        return 0
    og_year = int(str(date)[-2:]) + 2000
    year = og_year if og_year <= 2010 else og_year - 100
    og_year = year
    return og_year

data['age'] = data.apply(lambda x: get_age(x['birth_date'], x['date_filed']), axis=1)
data = data.drop(['birth_date'], axis = 1) 

Cleaning age field

In [59]:
from datetime import timedelta, date
import numpy as np

# Delete these row indexes from dataFrame. age is outlier
indexNames = data[ data['age'] >= 78 ].index
data.drop(indexNames , inplace=True)

data['age'] = data['age'].replace(0, np.nan) # to avoid counting in the zeroes
data['age'] = data['age'].replace(np.nan, data['age'].mean())

data.head()

,state_code,num_employee_code,naics_code,inst_type_code,sex,date_filed,closure_code,basis_code,issue_code,litigation_case_type,age
0,TX,C,nan,E,F,06/09/10,M3,GF,S5,NaN,39.00
1,NV,D,nan,E,NaN,05/11/10,M3,AC,D2,NaN,44.00
2,MA,NaN,"446,110.00",E,F,02/10/10,M3,OR,W1,NaN,29.00
3,PA,B,nan,G,F,11/03/09,M3,TP,T2,NaN,53.00
4,CA,A,nan,G,M,03/17/10,M3,VB,R6,NaN,49.00


In [60]:
data["litigation_case_type"].fillna("No Litigation", inplace = True)
data["naics_code"].fillna("No NAICS", inplace = True)
data["state_code"].fillna(data["state_code"].mode()[0], inplace = True)
data["sex"].fillna(data["sex"].mode()[0], inplace = True)
data["num_employee_code"].fillna(data["num_employee_code"].mode()[0], inplace = True)
# drop rows with no basis_code OR no closure_code.
data = data.dropna()
data.isna().sum()

state_code              0
num_employee_code       0
naics_code              0
inst_type_code          0
sex                     0
date_filed              0
closure_code            0
basis_code              0
issue_code              0
litigation_case_type    0
age                     0
dtype: int64

## Fix class label
Change values of closure code to their appropriate class of 0, 1, and 2 as described in "Grouping of Classes".

In [61]:
to_replace = {"M3" : 0, "M5" : 0, "X7" : 0, "M1" : 1, "M2" : 1, "M4" : 1, "X1" : 1, "X3" : 2, "X9" : 2, "Y1" : 2, "N2" : 2, "Y2" : 2, "X2" : 2}
data["closure_code"].replace(to_replace, inplace = True)
unusedClass = data[ data["closure_code"] == 2].index
data.drop(unusedClass , inplace=True)